In [70]:
import torch
import pandas as pd
import numpy as np
import pickle
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN
from pyclustering.cluster import dbscan, optics
import os
import sys
ROOT = "./"
sys.path.append(ROOT + "lib")
sys.path.append(ROOT + "models")

from lightsource import LightSource
from helpers import to_inference_input
from transformer import FluxAnomalyPredictionTF

modelid = "2769"

with open(ROOT + f"state_dicts/model{modelid}.pkl", "rb") as f:
    params = pickle.load(f)
print(params)
model = FluxAnomalyPredictionTF(params["resolution"], params["dropout"], bn=params["batchnorm"], features=params["features"], residual=params["residual"])
with open(ROOT + f"state_dicts/model{modelid}.pt", "rb") as f:
  model.load_state_dict(torch.load(f))


model.eval()
TEST = True
BATCHSIZE = 10

{'resolution': 20, 'dropout': 0.15, 'batchnorm': False, 'features': 3, 'residual': 0}


In [71]:
object_key = {}
row_key = {}
raw = pd.DataFrame()

if TEST:
  with open(ROOT + "cached_data/test_buckets.pkl", "rb") as f:
    test_buckets = pickle.load(f)

  for kind in test_buckets:
    for obj in test_buckets[kind]:
      tbl = obj.get_pandas()
      ra = tbl["ra"].to_numpy()
      dec = tbl["dec"].to_numpy()
      center = (np.mean(ra), np.mean(dec))
      object_key[center] = kind

      start = len(raw)
      end = start + len(tbl) - 1
      raw = pd.concat((raw, tbl), ignore_index=True)
      for i in range(start,end):
        row_key[i] = kind

  raw.insert(len(raw.columns), "idx", list(range(len(raw))), True) #indexing

else:
  pass
raw

,ra,dec,clon,clat,sigra,sigdec,sigradec,w1mpro,w1sigmpro,w1snr,...,tmass_key,j_m_2mass,j_msig_2mass,h_m_2mass,h_msig_2mass,k_m_2mass,k_msig_2mass,dist,angle,idx
0,89.561502,-66.324300,05h58m14.76s,-66d19m27.48s,0.0560,0.0543,0.0097,12.091,0.024,45.7,...,120700123,12.125,0.0256,12.112,0.0268,12.125,0.0229,0.000000,90.000002,0
1,89.561506,-66.324300,05h58m14.76s,-66d19m27.48s,0.0534,0.0634,-0.0132,12.017,0.024,45.0,...,120700123,12.125,0.0256,12.112,0.0268,12.125,0.0229,0.008132,82.401129,1
2,89.561502,-66.324303,05h58m14.76s,-66d19m27.49s,0.0526,0.0566,0.0176,12.041,0.023,47.7,...,120700123,12.125,0.0256,12.112,0.0268,12.125,0.0229,0.009720,161.360242,2
3,89.561507,-66.324299,05h58m14.76s,-66d19m27.48s,0.0574,0.0517,0.0281,12.069,0.022,49.7,...,120700123,12.125,0.0256,12.112,0.0268,12.125,0.0229,0.011500,66.371412,3
4,89.561491,-66.324300,05h58m14.76s,-66d19m27.48s,0.0517,0.0556,-0.0090,12.017,0.023,46.4,...,120700123,12.125,0.0256,12.112,0.0268,12.125,0.0229,0.013040,268.415031,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29232,160.598369,-82.037651,10h42m23.61s,-82d02m15.54s,0.0216,0.0256,0.0023,6.437,0.020,53.8,...,761419411,7.131,0.0302,6.941,0.0362,6.904,0.0380,1.714190,216.835051,29232
29233,160.600575,-82.037750,10h42m24.14s,-82d02m15.90s,0.0419,0.0427,-0.0073,6.849,0.018,59.7,...,761419411,7.131,0.0302,6.941,0.0362,6.904,0.0380,1.730603,177.597117,29233
29234,160.599742,-82.037760,10h42m23.94s,-82d02m15.94s,0.0268,0.0235,0.0110,6.592,0.018,59.7,...,761419411,7.131,0.0302,6.941,0.0362,6.904,0.0380,1.796313,191.004405,29234
29235,160.596851,-82.037453,10h42m23.24s,-82d02m14.83s,0.0199,0.0173,0.0054,6.592,0.012,91.7,...,761419411,7.131,0.0302,6.941,0.0362,6.904,0.0380,1.902030,249.767662,29235


Clustering and collecting objects

In [73]:
clstr_tbl = raw[["ra", "dec"]].to_numpy().tolist() # Careful of order
DBSCAN = optics.optics(clstr_tbl, 0.75/3600, 12, ccore=False)
DBSCAN.process()
clusters = DBSCAN.get_clusters()

In [83]:
classkey = ["null", "nova", "pulsator", "transit"]
for cluster in clusters:
    objtable=raw.iloc[cluster]
    tnsr = LightSource(objtable, "").to_tensor()
    inp = to_inference_input(tnsr).to(torch.float32)
    out = model(inp)
    guess = classkey[torch.argmax(out)]
    print(out)
    print(torch.argmax(out))
    print(guess)
    
        

tensor([[-1.3306, -1.6157, -1.2166, -1.4243]], grad_fn=<LogSoftmaxBackward0>)
tensor(2)
pulsator
tensor([[-1.3302, -1.6151, -1.2182, -1.4232]], grad_fn=<LogSoftmaxBackward0>)
tensor(2)
pulsator
tensor([[-1.3296, -1.6146, -1.2199, -1.4222]], grad_fn=<LogSoftmaxBackward0>)
tensor(2)
pulsator
tensor([[-1.3298, -1.6148, -1.2193, -1.4225]], grad_fn=<LogSoftmaxBackward0>)
tensor(2)
pulsator
tensor([[-1.3303, -1.6153, -1.2177, -1.4235]], grad_fn=<LogSoftmaxBackward0>)
tensor(2)
pulsator
tensor([[-1.3317, -1.6165, -1.2135, -1.4261]], grad_fn=<LogSoftmaxBackward0>)
tensor(2)
pulsator
tensor([[-1.3304, -1.6153, -1.2175, -1.4236]], grad_fn=<LogSoftmaxBackward0>)
tensor(2)
pulsator
tensor([[-1.3307, -1.6161, -1.2157, -1.4250]], grad_fn=<LogSoftmaxBackward0>)
tensor(2)
pulsator
tensor([[-1.3313, -1.6160, -1.2151, -1.4251]], grad_fn=<LogSoftmaxBackward0>)
tensor(2)
pulsator
tensor([[-1.3298, -1.6148, -1.2193, -1.4225]], grad_fn=<LogSoftmaxBackward0>)
tensor(2)
pulsator
tensor([[-1.3296, -1.6147, -1.

In [ ]:
objects = [LightSource(tbl) for tbl in objects]
feed = [to_inference_input(obj.to_tensor()) for obj in objects]

for obj in feed:
    
    